In [4]:
import cv2
import numpy as np
import imutils

CLOVE_MIN_AREA = 500
AFFECTED_AREA_THRESHOLD = 0.3

GOOD_CLOVE_COLOR_LOWER = np.array([10, 40, 40])
GOOD_CLOVE_COLOR_UPPER = np.array([30, 255, 180])

BAD_CLOVE_COLOR_LOWER = np.array([35, 50, 50])
BAD_CLOVE_COLOR_UPPER = np.array([90, 255, 200])

def resize_image(image, width, height):
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

def detect_clove_quality(image_path, reference_path):
    image = cv2.imread(image_path)
    reference_image = cv2.imread(reference_path, 0)
    if image is None or reference_image is None:
        print("Error: Input or reference image not found.")
        return

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    edges = cv2.Canny(gray, 50, 150)
    reference_edges = cv2.Canny(reference_image, 50, 150)
    combined_edges = cv2.bitwise_or(edges, reference_edges)

    kernel = np.ones((5, 5), np.uint8)
    combined_edges = cv2.morphologyEx(combined_edges, cv2.MORPH_CLOSE, kernel)

    cnts = cv2.findContours(combined_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    output_image = image.copy()

    total_cloves = 0
    bad_quality_cloves = 0

    for i, c in enumerate(cnts):
        area = cv2.contourArea(c)
        if area > CLOVE_MIN_AREA:
            total_cloves += 1
            single_clove_mask = np.zeros(gray.shape, dtype=np.uint8)
            cv2.drawContours(single_clove_mask, [c], -1, 255, thickness=cv2.FILLED)

            clove_hsv = cv2.bitwise_and(hsv, hsv, mask=single_clove_mask)
            good_clove_mask = cv2.inRange(clove_hsv, GOOD_CLOVE_COLOR_LOWER, GOOD_CLOVE_COLOR_UPPER)
            bad_clove_mask = cv2.inRange(clove_hsv, BAD_CLOVE_COLOR_LOWER, BAD_CLOVE_COLOR_UPPER)

            good_area = cv2.countNonZero(good_clove_mask)
            bad_area = cv2.countNonZero(bad_clove_mask)
            total_area = good_area + bad_area

            affected_ratio = bad_area / total_area if total_area > 0 else 0
            print(f"Clove {i}: Good Area = {good_area}, Bad Area = {bad_area}, Affected Ratio = {affected_ratio:.2f}")

            if affected_ratio > AFFECTED_AREA_THRESHOLD:
                bad_quality_cloves += 1
                label = "Bad Quality"
                color = (0, 0, 255)
            else:
                label = "Good Quality"
                color = (0, 255, 0)

            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(output_image, (x, y), (x + w, y + h), color, 2)
            cv2.putText(output_image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            # Debugging: Visualize masks
            cv2.imshow(f"Good Clove Mask {i}", good_clove_mask)
            cv2.imshow(f"Bad Clove Mask {i}", bad_clove_mask)

    print(f"Total Cloves: {total_cloves}")
    print(f"Bad Quality Cloves: {bad_quality_cloves}")
    print(f"Good Quality Cloves: {total_cloves - bad_quality_cloves}")

    grading_text = f"Total: {total_cloves}, Bad: {bad_quality_cloves}, Good: {total_cloves - bad_quality_cloves}"
    cv2.putText(output_image, grading_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow("Final Output", resize_image(output_image, 800, 600))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test paths
image_path = r"C:\Users\vidya\OneDrive\Documents\Spiceup-SCM\Phase 2\Clove-L\train\images\asdas_jpeg_jpg.rf.7ddcda98ac3a5296864bbaf735898fc2.jpg"
reference_path = r"C:\Users\vidya\OneDrive\Documents\Spiceup-SCM\Phase 2\Clove-L\train\images\WhatsApp-Image-2023-05-08-at-10-38-26-1-_jpeg_jpg.rf.1e59aa2908d04f89484a8a03cbbd3319.jpg"
detect_clove_quality(image_path, reference_path)


Clove 0: Good Area = 4256, Bad Area = 472, Affected Ratio = 0.10
Clove 3: Good Area = 2852, Bad Area = 452, Affected Ratio = 0.14
Clove 6: Good Area = 2533, Bad Area = 484, Affected Ratio = 0.16
Clove 7: Good Area = 2950, Bad Area = 481, Affected Ratio = 0.14
Clove 18: Good Area = 2760, Bad Area = 743, Affected Ratio = 0.21
Clove 19: Good Area = 0, Bad Area = 300, Affected Ratio = 1.00
Total Cloves: 6
Bad Quality Cloves: 1
Good Quality Cloves: 5
